In [ ]:
#https://www.bioconductor.org/packages/devel/bioc/vignettes/zellkonverter/inst/doc/zellkonverter.html

In [1]:
library(SeuratDisk)
library(tidyverse)
library(Seurat)

library(nebula)
library(fixest)
library(glmGamPoi)
library(limma)
library(edgeR)
library(SingleCellExperiment)
library(data.table)

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Attaching SeuratObject


Attaching package: ‘glmGamPoi’


The following object is masked from ‘package:dplyr’:

    vars


The following object is masked from ‘package:ggplot2’:

    vars


Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading req

In [2]:
selectCol <- function(mat, j.col){    
    x.col.dense <- rep(0,nrow(mat))
    p.begin <- mat@p[j.col]+1
    p.end <- mat@p[j.col+1]
    i.col <- mat@i[p.begin:p.end]+1 # i counts from 0
    x.col <- mat@x[p.begin:p.end]
    x.col.dense[i.col] <- x.col
    return(x.col.dense)
    }

selectCols <- function(mat, j.cols){    
    return(sapply(j.cols, selectCol, mat=mat))
    }

fixest.mult <- function(formula, count, df){
    df.result <- data.frame(matrix(nrow=0, ncol=4))
    colnames(df.result) <- c('Estimate', 'Std. Error', 't value', 'Pr(>|t|)')
    for (j in 1:ncol(count)){
        df$y <- selectCol(count, j)
        fit <- fixest::fepois(formula, vcov='hetero', data=df)
        df.result[colnames(count)[j],] <- coeftable(fit)['tx_cell',] # fixed effect o/x 에 따라 다르게 들어가야함
        } 
    return(df.result)
    }

nebula.mult <- function(formula, count, df){
    pred <- model.matrix(formula, data=df)
    sid <- df$id
    fit.nebula <- nebula::nebula(
        count,
        sid,
        pred=pred,
        cpc=0,
        mincp=0
        )
    fit.result <- fit.nebula$summary
    rownames(fit.result) <- fit.result$gene
    return(
            fit.result %>%
            mutate(
                Estimate=logFC_tx_cell,
                'Std. Error'=se_tx_cell,
                't value'=logFC_tx_cell/se_tx_cell,
                'Pr(>|t|)'=p_tx_cell
                ) %>%
            select(Estimate, 'Std. Error', 't value', 'Pr(>|t|)')
        )
    }

glmgp.mult <- function(formula, count, df){
    sce.obj <- SingleCellExperiment::SingleCellExperiment(list(counts=count), colData=df)
    sce.pb <- glmGamPoi::pseudobulk(
        sce.obj,
        group_by=vars(id, tx_cell),
        verbose=FALSE
        )

    fit <- glmGamPoi::glm_gp(sce.pb, design=~1+tx_cell)
    test <- glmGamPoi::test_de(fit, reduced_design=~1)
    
    beta <- fit$Beta[,'tx_cell']
    pval <- test$pval
    tval <- qnorm(1-pval/2) * sign(beta)
    se <- beta/tval
    result <- cbind(beta, se, tval, pval)
    colnames(result) <- c('Estimate', 'Std. Error', 't value', 'Pr(>|t|)')
    return(result)
    }

glmgp.cell.mult <- function(formula, count, df){
    sce.obj <- SingleCellExperiment::SingleCellExperiment(list(counts=count), colData=df)
    fit <- glmGamPoi::glm_gp(sce.obj, design=~1+tx_cell, on_disk=FALSE, size_factors=FALSE)
    test <- glmGamPoi::test_de(fit, reduced_design=~1)
    
    beta <- fit$Beta[,'tx_cell']
    pval <- test$pval
    tval <- qnorm(1-pval/2) * sign(beta)
    se <- beta/tval
    result <- cbind(beta, se, tval, pval)
    colnames(result) <- c('Estimate', 'Std. Error', 't value', 'Pr(>|t|)')
    return(result)
    }

edger.mult <- function(formula, count, df){
    sce.obj <- SingleCellExperiment::SingleCellExperiment(list(counts=count), colData=df)
    sce.pb <- glmGamPoi::pseudobulk(
        sce.obj,
        group_by=vars(id, tx_cell),
        verbose=FALSE
        )

    design <- model.matrix(~1+tx_cell, data=colData(sce.pb))
    edger.obj <- edgeR::DGEList(counts(sce.pb))
    edger.obj <- edgeR::estimateDisp(edger.obj, design)
    fit <- edgeR::glmQLFit(y=edger.obj, design=design)
    test <- edgeR::glmTreat(fit, coef=2)

    beta <- test$coefficients[,'tx_cell']
    pval <- test$table[,'PValue']
    tval <- qnorm(1-pval/2) * sign(beta)
    se <- beta/tval

    result <- cbind(beta, se, tval, pval)
    colnames(result) <- c('Estimate', 'Std. Error', 't value', 'Pr(>|t|)')
    
    return(result)
    }

limma.mult <- function(formula, count, df){
    sce.obj <- SingleCellExperiment::SingleCellExperiment(list(counts=count), colData=df)
    sce.pb <- glmGamPoi::pseudobulk(
        sce.obj,
        group_by=vars(id, tx_cell),
        verbose=FALSE
        )

    design <- model.matrix(~1+tx_cell, data=colData(sce.pb))
    edger.obj <- edgeR::DGEList(counts(sce.pb))
    v <- limma::voom(edger.obj, design)
    vfit <- limma::lmFit(v, design)
    efit <- limma::eBayes(vfit)
    
    beta <- efit$coefficients[,'tx_cell'] * log(2)
    pval <- efit$p.value[,'tx_cell']
    tval <- qnorm(1-pval/2) * sign(beta)
    se <- beta/tval

    result <- cbind(beta, se, tval, pval)
    colnames(result) <- c('Estimate', 'Std. Error', 't value', 'Pr(>|t|)')
    
    return(result)
    }

In [3]:
obj <- LoadH5Seurat("../datasets_pseq/Perturb-seq/data/HuTcellsCRISPRaPerturbSeq_Re-stimulated.h5Seurat")

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding miscellaneous information for RNA

Initializing SCT with data

Adding counts for SCT

Adding scale.data for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding graph SCT_nn

Adding graph SCT_snn

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [4]:
cytokine_genes <- read_tsv("../datasets_pseq/Perturb-seq/data/GO_0005125_Cytokines.txt", col_names = F)
cytokine_genes <- cytokine_genes$X3 %>% unique() %>% sort()
granzyme_genes <- c("GZMA", "GZMB", "GZMH", "GZMK", "GZMM", "PRF1", "GNLY", "THBS1")
detected_features <- rownames(obj)
cytokines_in_data <- cytokine_genes[cytokine_genes %in% detected_features]
granzyme_genes_in_data <- granzyme_genes[granzyme_genes %in% detected_features]
test_genes <- c(cytokines_in_data, granzyme_genes_in_data)

Rows: 320 Columns: 15
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (13): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X14
dbl  (1): X15
lgl  (1): X13

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
length(test_genes)
test_genes

[1] 173

[1] "AIMP1"     "AREG"      "BMP1"      "BMP10"     "BMP2"      "BMP4"     
  [7] "BMP6"      "BMP7"      "BMP8A"     "BMP8B"     "C17orf99"  "C1QTNF4"  
 [13] "C5"        "CCL1"      "CCL14"     "CCL16"     "CCL17"     "CCL18"    
 [19] "CCL20"     "CCL22"     "CCL23"     "CCL25"     "CCL27"     "CCL28"    
 [25] "CCL3"      "CCL3L1"    "CCL4"      "CCL5"      "CD40LG"    "CD70"     
 [31] "CKLF"      "CLCF1"     "CMTM1"     "CMTM2"     "CMTM3"     "CMTM7"    
 [37] "CMTM8"     "CNTF"      "CRLF1"     "CSF1"      "CSF2"      "CTF1"     
 [43] "CXCL10"    "CXCL11"    "CXCL13"    "CXCL14"    "CXCL16"    "CXCL2"    
 [49] "CXCL3"     "CXCL8"     "CXCL9"     "EBI3"      "EDN1"      "FAM3B"    
 [55] "FAM3C"     "FASLG"     "FGF2"      "FLT3LG"    "GDF10"     "GDF11"    
 [61] "GDF15"     "GDF5"      "GDF7"      "GDF9"      "GPI"       "GREM1"    
 [67] "GREM2"     "GRN"       "HMGB1"     "IFNB1"     "IFNE"      "IFNG"     
 [73] "IFNL1"     "IL10"      "IL11"      "IL12A"     "IL12B"     "IL13"     
 [79] "IL15"      "IL16"      "IL17A"     "IL17C"     "IL17D"     "IL17F"    
 [85] "IL18"      "IL19"      "IL1A"      "IL1B"      "IL1RN"     "IL2"      
 [91] "IL20"      "IL21"      "IL22"      "IL23A"     "IL24"      "IL26"     
 [97] "IL3"       "IL31"      "IL32"      "IL34"      "IL36A"     "IL36G"    
[103] "IL37"      "IL4"       "IL5"       "IL6"       "IL7"       "IL9"      
[109] "INHBA"     "INHBB"     "INHBC"     "INHBE"     "KITLG"     "LEFTY1"   
[115] "LIF"       "LTA"       "LTB"       "MIF"       "MSMP"      "MSTN"     
[121] "NAMPT"     "NODAL"     "NOG"       "NRG1"      "NRP1"      "OSM"      
[127] "SCG2"      "SCGB3A1"   "SECTM1"    "SPP1"      "TGFB1"     "TGFB2"    
[133] "TGFB3"     "THNSL2"    "TIMP1"     "TNF"       "TNFRSF11B" "TNFSF10"  
[139] "TNFSF11"   "TNFSF12"   "TNFSF13"   "TNFSF13B"  "TNFSF14"   "TNFSF15"  
[145] "TNFSF4"    "TNFSF8"    "TNFSF9"    "TSLP"      "VEGFA"     "WNT1"     
[151] "WNT10A"    "WNT10B"    "WNT11"     "WNT16"     "WNT2B"     "WNT3"     
[157] "WNT4"      "WNT5B"     "WNT6"      "WNT7A"     "WNT7B"     "WNT8B"    
[163] "WNT9A"     "XCL1"      "XCL2"      "GZMA"      "GZMB"      "GZMH"     
[169] "GZMK"      "GZMM"      "PRF1"      "GNLY"      "THBS1"

In [6]:
# gene column에 있는 거랑 NT랑 subset해서 1:1로 비교하는 구조
targetgenes <- obj@meta.data %>%
  dplyr::filter(gene != "NO-TARGET") %>%
  pull(gene) %>%
  as.character() %>%
  unique()
length(targetgenes)
targetgenes

[1] 69

[1] "ABCB10"    "AKAP12"    "ALX4"      "APOBEC3C"  "APOBEC3D"  "APOL2"    
 [7] "ARHGDIB"   "BICDL2"    "CBY1"      "CD2"       "CD247"     "CD27"     
[13] "CD28"      "CEACAM1"   "CNR2"      "DEF6"      "DEPDC7"    "EMP1"     
[19] "EMP3"      "EOMES"     "FOSB"      "FOSL1"     "FOXD2"     "FOXL2NB"  
[25] "FOXQ1"     "GATA3"     "GRAP"      "IFNG"      "IKZF3"     "IL1R1"    
[31] "IL2"       "IL2RB"     "IL2RG"     "IL9R"      "INPPL1"    "ITPKA"    
[37] "JMJD1C"    "LAT"       "LAT2"      "LCP2"      "LHX4"      "LHX6"     
[43] "LTBR"      "MAP4K1"    "MGST3"     "MUC1"      "NLRC3"     "NOTCH1"   
[49] "OTUD7A"    "OTUD7B"    "P2RY14"    "PAPOLG"    "PIK3AP1"   "PLCG2"    
[55] "PRDM13"    "PRKD2"     "RAC2"      "RELA"      "SLA2"      "TAGAP"    
[61] "TBX21"     "TNFRSF12A" "TNFRSF1A"  "TNFRSF1B"  "TNFRSF9"   "TRAF3IP2" 
[67] "TRIM21"    "VAV1"      "WT1"

In [215]:
# adopted from the authors
target_gene_overexpression <- function(obj, 
                                       Gene, # perturbed gene
                                       GeneSet = test_genes
                                      ) { # tested gene
    
    # extract information
    obj <- subset(obj, gene %in% c("NO-TARGET", Gene))
    cnt <- GetAssayData(object = obj, slot = "counts")
    col.data <- obj[[c('gene', 'donor')]]
    colnames(col.data) <- c('tx_cell', 'donor_id')
    col.data$tx_cell <- ifelse(col.data$tx_cell == 'NO-TARGET', 0, 1)
    
    # sort by donor id (nebula requires it)
    col.data <- col.data %>% arrange(donor_id)
    col.data$id <- col.data$donor_id
    cnt <- cnt[,rownames(col.data)]
    cnt <- cnt[GeneSet,]
    cnt <- cnt[rowMeans(cnt) > 0.01,]
    #sce.obj <- SingleCellExperiment::SingleCellExperiment(list(counts=cnt), colData=col.data)

    # iterate over methods

    if(TRUE){
    func.list <- list(nebula.mult, glmgp.mult, edger.mult, limma.mult, glmgp.cell.mult, fixest.mult)
    data.list <- list(cnt, cnt, cnt, cnt, cnt, t(cnt))
    form.list <- list(
        as.formula('~tx_cell'),
        as.formula('~tx_cell'),
        as.formula('~tx_cell'),
        as.formula('~tx_cell'),
        as.formula('~tx_cell'),
        as.formula('y~tx_cell | donor_id')
        )

    result.list <- list()
    for (i in 1:length(func.list)){
        result.list[[i]] <- func.list[[i]](
            form.list[[i]],
            data.list[[i]],
            col.data
            )
        }
    }
    
    return(result.list)
}

In [217]:
# genes with large mean are added for proper normalization
result <- lapply(targetgenes, 
                 target_gene_overexpression, 
                 obj=obj, 
                 GeneSet=unique(c(test_genes,rownames(cnt)[(rowMeans2(cnt) > 1)]))
                )
saveRDS(result, 'schdmidt.result.rds')

Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2099  cells.
Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2070  cells.
Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2159  cells.
Remove  0  genes having low expression.
Analyzing  940  genes with  2  subjects and  2186  cells.
Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2177  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2167  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2250  cells.
Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2290  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2393  cells.
Remove  0  genes having low expression.
Analyzing  940  genes with  2  subjects and  2269  cells.
Remove  0  genes hav

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) removed because of only 0 outcomes.

NOTE: 1 fixed-effect (1 observation) r

Remove  0  genes having low expression.
Analyzing  940  genes with  2  subjects and  2135  cells.
Remove  0  genes having low expression.
Analyzing  941  genes with  2  subjects and  2112  cells.
Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2076  cells.
Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2437  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2467  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2128  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2310  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2460  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2291  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2339  cells.
Remove  0  genes hav

NOTE: 1 fixed-effect (1,189 observations) removed because of only 0 outcomes.



Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2297  cells.
Remove  0  genes having low expression.
Analyzing  937  genes with  2  subjects and  2549  cells.
Remove  0  genes having low expression.
Analyzing  939  genes with  2  subjects and  2402  cells.
Remove  0  genes having low expression.
Analyzing  940  genes with  2  subjects and  2051  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2318  cells.
Remove  0  genes having low expression.
Analyzing  941  genes with  2  subjects and  2317  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2249  cells.
Remove  0  genes having low expression.
Analyzing  937  genes with  2  subjects and  2188  cells.
Remove  0  genes having low expression.
Analyzing  938  genes with  2  subjects and  2422  cells.
Remove  0  genes having low expression.
Analyzing  937  genes with  2  subjects and  2149  cells.
Remove  0  genes hav

In [7]:
result <- readRDS('schdmidt.result.rds')
for (i in 1:length(result)){
    for (j in 1:length(result[[i]])){
        result[[i]][[j]] <- as.data.frame(result[[i]][[j]])
        result[[i]][[j]]['gene'] <- rownames(result[[i]][[j]])
        }
    names(result[[i]]) <- c('NB GLMM', 'glmGamPoi (Pb)', 'edgeR (Pb)', 'limma (Pb)', 'glmGamPoi (cell)', 'fixest (cell)')
    }

In [8]:
pt.vec <- targetgenes
dt.list <- list()
for (i in 1:length(pt.vec)){
    dt.list[[pt.vec[i]]] <- rbindlist(result[[i]], idcol='method')
    }

In [22]:
df.hm <- rbindlist(dt.list, idcol='perturb_gene') %>% 
    filter(gene %in% test_genes) %>%
    mutate(gene_perturb = paste(gene, perturb_gene, sep='_')) %>%
    select(gene_perturb, method, `Pr(>|t|)`) %>% 
    pivot_wider(names_from=method, values_from=`Pr(>|t|)`)
df.hm <- as.data.frame(df.hm[,2:ncol(df.hm)])
df.hm[!is.finite(df.hm[,1]),] <- 1
mat <- as.matrix(df.hm)
pval <- 0.05 / 4807
mat <- (mat < pval)
cor.mat <- round((t(mat) %*% mat)/nrow(mat) * 100,2)
name.ord <- c('glmGamPoi (Pb)', 'edgeR (Pb)', 'limma (Pb)', 'glmGamPoi (cell)', 'fixest (cell)', 'NB GLMM')
write.csv(cor.mat[name.ord, name.ord], 'schmidt.pow.csv')
cor.mat[name.ord, name.ord]


,glmGamPoi (Pb),edgeR (Pb),limma (Pb),glmGamPoi (cell),fixest (cell),NB GLMM
glmGamPoi (Pb),0,0.0,0,0.00,0.00,0.00
edgeR (Pb),0,0.1,0,0.10,0.10,0.10
limma (Pb),0,0.0,0,0.00,0.00,0.00
glmGamPoi (cell),0,0.1,0,10.76,8.97,9.92
fixest (cell),0,0.1,0,8.97,11.34,9.03
NB GLMM,0,0.1,0,9.92,9.03,10.59


In [21]:
df.hm <- rbindlist(dt.list, idcol='perturb_gene') %>% 
    filter(gene %in% test_genes) %>%
    mutate(gene_perturb = paste(gene, perturb_gene, sep='_')) %>%
    select(gene_perturb, method, `Pr(>|t|)`) %>% 
    pivot_wider(names_from=method, values_from=`Pr(>|t|)`)
df.hm <- as.data.frame(df.hm[,2:ncol(df.hm)])
df.hm[!is.finite(df.hm[,1]),] <- 1
mat <- as.matrix(df.hm)
cor.mat <- cor(mat, method = c("spearman"))
name.ord <- c('glmGamPoi (Pb)', 'edgeR (Pb)', 'limma (Pb)', 'glmGamPoi (cell)', 'fixest (cell)', 'NB GLMM')
write.csv(cor.mat[name.ord, name.ord], 'schmidt.sp.csv')
cor.mat[name.ord, name.ord]


,glmGamPoi (Pb),edgeR (Pb),limma (Pb),glmGamPoi (cell),fixest (cell),NB GLMM
glmGamPoi (Pb),1.0000000,0.9667525,0.9279869,0.8951176,0.9219876,0.8908991
edgeR (Pb),0.9667525,1.0000000,0.8980146,0.8467224,0.8924995,0.8487321
limma (Pb),0.9279869,0.8980146,1.0000000,0.8373444,0.8560973,0.8476330
glmGamPoi (cell),0.8951176,0.8467224,0.8373444,1.0000000,0.9547167,0.9645181
fixest (cell),0.9219876,0.8924995,0.8560973,0.9547167,1.0000000,0.9335277
NB GLMM,0.8908991,0.8487321,0.8476330,0.9645181,0.9335277,1.0000000
